In [1]:
import pandas as pd
import polars as pl
import numpy as np
import lightgbm as lgb
import optuna
import os

In [10]:
predicciones_path = 'C:/Eugenio/Maestria/DMEyF/exp/competencia_3/v009/'

In [11]:
predicciones_v05 = pd.read_csv(predicciones_path + 'predicciones_modelos_v005.csv')
predicciones_v07 = pd.read_csv(predicciones_path + 'predicciones_modelos_v007.csv')
predicciones_v08 = pd.read_csv(predicciones_path + 'predicciones_modelos_v008.csv')

In [12]:
predicciones = pd.merge(predicciones_v05, predicciones_v07, on='numero_de_cliente', how='inner')

predicciones = pd.merge(predicciones, predicciones_v08, on='numero_de_cliente', how='inner')

In [14]:
predicciones['promedio'] = predicciones.drop(columns=['numero_de_cliente']).mean(axis=1)

In [16]:
# ordeno de mayor probabilidad de baja a menor
idx = np.argsort(predicciones['promedio'])[::-1]
predicciones = predicciones.iloc[idx]

In [17]:
predicciones.head()

,numero_de_cliente,pred_lgm_v005_1149,pred_lgm_v005_4836,pred_lgm_v005_9443,pred_lgm_v005_7483,pred_lgm_v005_3492,pred_lgm_v005_6287,pred_lgm_v005_830,pred_lgm_v005_3294,pred_lgm_v005_5932,...,pred_lgm_v008_4836,pred_lgm_v008_9443,pred_lgm_v008_7483,pred_lgm_v008_3492,pred_lgm_v008_6287,pred_lgm_v008_830,pred_lgm_v008_3294,pred_lgm_v008_5932,pred_lgm_v008_13,promedio
116093,939475623,0.955374,0.964811,0.966366,0.967273,0.971096,0.958927,0.965807,0.968174,0.961317,...,0.999936,0.999867,0.999961,0.999919,0.999981,0.999934,0.999908,0.999942,0.999925,0.987854
129268,686309464,0.935251,0.940534,0.936892,0.932487,0.947191,0.938647,0.945878,0.943610,0.930579,...,0.999964,0.999988,0.999997,0.999985,0.999996,0.999989,0.999991,0.999990,0.999974,0.979682
70730,564497200,0.912695,0.931304,0.930893,0.914322,0.938628,0.928445,0.924184,0.923870,0.917289,...,0.999947,0.999915,0.999856,0.999702,0.999902,0.999912,0.999976,0.999922,0.999941,0.975100
89057,1160314594,0.912671,0.930612,0.920524,0.921071,0.921972,0.921880,0.920403,0.913279,0.928205,...,0.999037,0.998736,0.999460,0.999635,0.999010,0.999309,0.999500,0.999201,0.999217,0.973289
18216,1144407016,0.921882,0.926338,0.924977,0.915418,0.917509,0.914123,0.912944,0.914293,0.925503,...,0.999824,0.999827,0.999760,0.999552,0.999792,0.999830,0.999809,0.999792,0.999920,0.972012


In [18]:
puntos_corte = np.linspace(10000, 12000, 25).astype(int)
puntos_corte

array([10000, 10083, 10166, 10250, 10333, 10416, 10500, 10583, 10666,
       10750, 10833, 10916, 11000, 11083, 11166, 11250, 11333, 11416,
       11500, 11583, 11666, 11750, 11833, 11916, 12000])

In [19]:
counter = 0

for corte in puntos_corte:
    
    envios = np.zeros(len(predicciones), dtype=int)
    envios[:corte] = 1
    predicciones['Predicted'] = envios
    
    output = predicciones[['numero_de_cliente', 'Predicted']]
        
    counter += 1
    
    file_name = f'results_v009_{counter}.csv'
    output_path = predicciones_path + file_name
    
    output.to_csv(output_path, index=False)
            
    print(f'{counter}. corte: {corte}, {output_path}')

1. corte: 10000, C:/Eugenio/Maestria/DMEyF/exp/competencia_3/v009/results_v009_1.csv
2. corte: 10083, C:/Eugenio/Maestria/DMEyF/exp/competencia_3/v009/results_v009_2.csv
3. corte: 10166, C:/Eugenio/Maestria/DMEyF/exp/competencia_3/v009/results_v009_3.csv
4. corte: 10250, C:/Eugenio/Maestria/DMEyF/exp/competencia_3/v009/results_v009_4.csv
5. corte: 10333, C:/Eugenio/Maestria/DMEyF/exp/competencia_3/v009/results_v009_5.csv
6. corte: 10416, C:/Eugenio/Maestria/DMEyF/exp/competencia_3/v009/results_v009_6.csv
7. corte: 10500, C:/Eugenio/Maestria/DMEyF/exp/competencia_3/v009/results_v009_7.csv
8. corte: 10583, C:/Eugenio/Maestria/DMEyF/exp/competencia_3/v009/results_v009_8.csv
9. corte: 10666, C:/Eugenio/Maestria/DMEyF/exp/competencia_3/v009/results_v009_9.csv
10. corte: 10750, C:/Eugenio/Maestria/DMEyF/exp/competencia_3/v009/results_v009_10.csv
11. corte: 10833, C:/Eugenio/Maestria/DMEyF/exp/competencia_3/v009/results_v009_11.csv
12. corte: 10916, C:/Eugenio/Maestria/DMEyF/exp/competencia_3